<a href="https://colab.research.google.com/github/ShabibSaleh/DPAM-Nutrition/blob/main/Step1_Data_preparation_DPAM_Population_weighted_coverage_of_health_services.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gspread google-auth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.33.0 which is incompatible.


In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
# Open a sheet from a spreadsheet by name
spreadsheet = gc.open('DPAM Population-weighted coverage of health services')

# If you have the URL, you can do it this way:
# spreadsheet = gc.open_by_url('Your_Spreadsheet_URL')

# Data preparation and Cleaning

## on-track and off-track countries dataset

In [3]:
# Get a worksheet by name
worksheet = spreadsheet.worksheet('On & off-track countries')

# Get all values from the worksheet
rows = worksheet.get_all_values()




In [4]:
#Convert data to pandas DataFrame (if needed)
import pandas as pd
on_off_track_df = pd.DataFrame.from_records(rows[1:],columns=rows[0])  # Skip header row

In [5]:
# Checking for any missing values and the data types in the dataframe
on_off_track_df.info()

# Summary statistics to understand numerical columns, although most data seems categorical
on_off_track_df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ISO3Code      200 non-null    object
 1   OfficialName  200 non-null    object
 2   Status.U5MR   200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB


,ISO3Code,OfficialName,Status.U5MR
count,200,200,200
unique,200,200,3
top,AFG,Afghanistan,Achieved
freq,1,1,134


In [6]:
# Step 1: Check for duplicates
duplicates = on_off_track_df.duplicated().sum()

# Step 2: Checking the unique values in the "Status.U5MR" column to ensure consistency
unique_status_values = on_off_track_df['Status.U5MR'].unique()

# Step 3: Renaming columns for clarity
on_off_track_df_cleaned = on_off_track_df.rename(columns={
    'ISO3Code': 'Country Code',
    'OfficialName': 'Country Name',
    'Status.U5MR': 'U5MR Status'
})

# Displaying the results
duplicates, unique_status_values, on_off_track_df_cleaned.head()


(0,
 array(['Acceleration Needed', 'Achieved', 'On Track'], dtype=object),
   Country Code Country Name          U5MR Status
 0          AFG  Afghanistan  Acceleration Needed
 1          AGO       Angola  Acceleration Needed
 2          AIA     Anguilla             Achieved
 3          ALB      Albania             Achieved
 4          AND      Andorra             Achieved)

In [7]:
# Updating the values in the 'U5MR Status' column
on_off_track_df_cleaned['U5MR Status'] = on_off_track_df_cleaned['U5MR Status'].replace({
    'Achieved': 'On-Track',
    'On Track': 'On-Track',
    'Acceleration Needed': 'Off-Track'
})

# Displaying the first few rows to confirm the changes
on_off_track_df_cleaned.head()


,Country Code,Country Name,U5MR Status
0,AFG,Afghanistan,Off-Track
1,AGO,Angola,Off-Track
2,AIA,Anguilla,On-Track
3,ALB,Albania,On-Track
4,AND,Andorra,On-Track


Extract on-track and off-track cleaned data countries dataset to new Google sheet

In [8]:
# Add a new sheet with a name and specify rows and columns
worksheet_title = 'on_off_track_df_cleaned'

ws = spreadsheet.worksheet(worksheet_title)

In [9]:
# Use the gspread_dataframe to set the DataFrame to the sheet
from gspread_dataframe import set_with_dataframe
# Set the DataFrame to the worksheet
set_with_dataframe(ws, on_off_track_df_cleaned)

# ANC4 & SAB Data

In [10]:
# Get a worksheet by name
ANC4_SAB_worksheet = spreadsheet.worksheet('ANC4 & SAB Data')

# Get all values from the worksheet
ANC4_SAB_rows = ANC4_SAB_worksheet.get_all_values()

In [11]:
# Checking the general information and structure of the ANC4 & SAB Data sheet
anc4_sab_df = pd.DataFrame.from_records(ANC4_SAB_rows[1:],columns=ANC4_SAB_rows[0])  # Skip header row
anc4_sab_df.info()

# Summary statistics for numerical columns
anc4_sab_df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Geographic area  291 non-null    object
 1   Indicator        291 non-null    object
 2   Sex              291 non-null    object
 3   2022             291 non-null    object
 4   2021             291 non-null    object
 5   2020             291 non-null    object
 6   2019             291 non-null    object
 7   2018             291 non-null    object
dtypes: object(8)
memory usage: 18.3+ KB


,Geographic area,Indicator,Sex,2022,2021,2020,2019,2018
count,291,291,291,291,291,291,291,291
unique,181,2,1,62,34,54,71,84
top,Latin America and the Caribbean,Skilled birth attendant - percentage of delive...,Female,-,-,-,-,-
freq,7,159,291,210,251,213,175,159


In [12]:
# Converting the column names to strings
anc4_sab_df.columns = anc4_sab_df.columns.astype(str)


year_columns = ['2022', '2021', '2020', '2019', '2018']
anc4_sab_df[year_columns] = anc4_sab_df[year_columns].apply(pd.to_numeric, errors='coerce')

# Step 3: Drop the "Sex" column as it is redundant
anc4_sab_df_cleaned = anc4_sab_df.drop(columns=['Sex'])

# Displaying the cleaned data to verify
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,2022,2021,2020,2019,2018
0,Afghanistan,Antenatal care 4+ visits - percentage of women...,NaN,NaN,27.6,NaN,20.9
1,Afghanistan,Skilled birth attendant - percentage of delive...,NaN,NaN,61.8,NaN,58.8
2,Africa,Antenatal care 4+ visits - percentage of women...,56.8,NaN,NaN,NaN,NaN
3,Africa,Skilled birth attendant - percentage of delive...,71.0,NaN,NaN,NaN,NaN
4,Albania,Antenatal care 4+ visits - percentage of women...,NaN,NaN,NaN,NaN,77.8


In [13]:
# Creating a new column 'Most Recent Coverage Estimate' which takes the most recent non-null value from 2018 to 2022
anc4_sab_df_cleaned['Most Recent Coverage Estimate'] = anc4_sab_df_cleaned[year_columns].bfill(axis=1).iloc[:, 0]

# Displaying the updated dataframe
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,2022,2021,2020,2019,2018,Most Recent Coverage Estimate
0,Afghanistan,Antenatal care 4+ visits - percentage of women...,NaN,NaN,27.6,NaN,20.9,27.6
1,Afghanistan,Skilled birth attendant - percentage of delive...,NaN,NaN,61.8,NaN,58.8,61.8
2,Africa,Antenatal care 4+ visits - percentage of women...,56.8,NaN,NaN,NaN,NaN,56.8
3,Africa,Skilled birth attendant - percentage of delive...,71.0,NaN,NaN,NaN,NaN,71.0
4,Albania,Antenatal care 4+ visits - percentage of women...,NaN,NaN,NaN,NaN,77.8,77.8


In [14]:
# Dropping the year columns as requested
anc4_sab_df_cleaned = anc4_sab_df_cleaned.drop(columns=year_columns)

# Displaying the cleaned dataframe with the year columns removed
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,Most Recent Coverage Estimate
0,Afghanistan,Antenatal care 4+ visits - percentage of women...,27.6
1,Afghanistan,Skilled birth attendant - percentage of delive...,61.8
2,Africa,Antenatal care 4+ visits - percentage of women...,56.8
3,Africa,Skilled birth attendant - percentage of delive...,71.0
4,Albania,Antenatal care 4+ visits - percentage of women...,77.8


In [15]:
# Replacing the values in the 'Indicator' column as requested
anc4_sab_df_cleaned['Indicator'] = anc4_sab_df_cleaned['Indicator'].replace({
    'Antenatal care 4+ visits - percentage of women (aged 15-49 years) attended at least four times during pregnancy by any provider': 'ANC4',
    'Skilled birth attendant - percentage of deliveries attended by skilled health personnel': 'SAB'
})

# Displaying the updated dataframe to confirm changes
anc4_sab_df_cleaned.head()


,Geographic area,Indicator,Most Recent Coverage Estimate
0,Afghanistan,ANC4,27.6
1,Afghanistan,SAB,61.8
2,Africa,ANC4,56.8
3,Africa,SAB,71.0
4,Albania,ANC4,77.8


In [25]:
# Creating a pivot table with 'Geographic area' as the index, 'Indicator' as columns, and summing 'Most Recent Coverage Estimate'
anc4_sab_pivot = anc4_sab_df_cleaned.pivot_table(
    index='Geographic area',
    columns='Indicator',
    values='Most Recent Coverage Estimate',
    aggfunc='sum'
).reset_index()

# Displaying the resulting pivot table
anc4_sab_pivot.head()


Indicator,Geographic area,ANC4,SAB
0,Afghanistan,27.6,61.8
1,Africa,56.8,71.0
2,Albania,77.8,99.8
3,Algeria,69.8,98.8
4,Americas,94.0,96.8


In [26]:
# Renaming the 'Geographic area' column to 'Country Name'
anc4_sab_pivot.rename(columns={'Geographic area': 'Country Name'}, inplace=True)

# Displaying the updated pivot table
anc4_sab_pivot.head()


Indicator,Country Name,ANC4,SAB
0,Afghanistan,27.6,61.8
1,Africa,56.8,71.0
2,Albania,77.8,99.8
3,Algeria,69.8,98.8
4,Americas,94.0,96.8


Extract ANC4 & SAB cleaned dataset to new Google sheet

In [27]:
# Add a new sheet with a name and specify rows and columns
worksheet_title = 'anc4_sab_df_cleaned'

ws = spreadsheet.worksheet(worksheet_title)

In [28]:
# Use the gspread_dataframe to set the DataFrame to the sheet
from gspread_dataframe import set_with_dataframe
# Set the DataFrame to the worksheet
set_with_dataframe(ws, anc4_sab_pivot)

# Population data

In [29]:
# Get a worksheet by name
population_worksheet = spreadsheet.worksheet('Population data')

# Get all values from the worksheet
population_rows = population_worksheet.get_all_values()

# Checking the general information and structure of the ANC4 & SAB Data sheet
population_df = pd.DataFrame.from_records(population_rows[1:],columns=population_rows[0])  # Skip header row
population_df.info()
# Selecting the relevant columns as per the user's request
selected_columns = [
    'Index',
    'Variant',
    'Region, subregion, country or area *',
    'Notes',
    'Location code',
    'ISO3 Alpha-code',
    'ISO2 Alpha-code',
    'SDMX code**',
    'Type',
    'Parent code',
    'Year',
    'Total Population, as of 1 January (thousands)',
    'Total Population, as of 1 July (thousands)',
    'Male Population, as of 1 July (thousands)',
    'Female Population, as of 1 July (thousands)',
    'Births (thousands)',
    'Births by women aged 15 to 19 (thousands)',
    'Crude Birth Rate (births per 1,000 population)',
    'Total Fertility Rate (live births per woman)',
    'Net Reproduction Rate (surviving daughters per woman)',
    'Mean Age Childbearing (years)',
    'Sex Ratio at Birth (males per 100 female births)',
    'Live Births Surviving to Age 1 (thousands)',
    'Under-Five Deaths, under age 5 (thousands)',
    'Under-Five Mortality (deaths under age 5 per 1,000 live births)'
]

# Selecting only these columns and dropping others
population_df_selected = population_df[selected_columns]

# Renaming the columns
population_df_selected.rename(columns={
    'Region, subregion, country or area *': 'Country Name',
    'ISO3 Alpha-code': 'Country Code'
}, inplace=True)

# Displaying the first few rows to confirm the changes
population_df_selected.head()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 25 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   Index                                                            237 non-null    object
 1   Variant                                                          237 non-null    object
 2   Region, subregion, country or area *                             237 non-null    object
 3   Notes                                                            237 non-null    object
 4   Location code                                                    237 non-null    object
 5   ISO3 Alpha-code                                                  237 non-null    object
 6   ISO2 Alpha-code                                                  237 non-null    object
 7   SDMX code**                                          

,Index,Variant,Country Name,Notes,Location code,Country Code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,Births (thousands),Births by women aged 15 to 19 (thousands),"Crude Birth Rate (births per 1,000 population)",Total Fertility Rate (live births per woman),Net Reproduction Rate (surviving daughters per woman),Mean Age Childbearing (years),Sex Ratio at Birth (males per 100 female births),Live Births Surviving to Age 1 (thousands),"Under-Five Deaths, under age 5 (thousands)","Under-Five Mortality (deaths under age 5 per 1,000 live births)"
0,1901,Medium,Burundi,,108,BDI,BI,108,Country/Area,910,...,440.0,37.0,34.1,5,2.2,31,103.00,426.0,22,51
1,1980,Medium,Comoros,,174,COM,KM,174,Country/Area,910,...,24.0,2.0,28.9,4,1.8,31,103.00,23.3,1,58
2,2059,Medium,Djibouti,,262,DJI,DJ,262,Country/Area,910,...,25.0,1.0,21.9,3,1.2,31,104.00,23.8,1,51
3,2138,Medium,Eritrea,,232,ERI,ER,232,Country/Area,910,...,105.0,14.0,28.4,4,1.7,29,103.00,102.4,4,38
4,2217,Medium,Ethiopia,,231,ETH,ET,231,Country/Area,910,...,3 928,450.0,31.8,4,1.8,30,106.00,3820.0,169,44


In [30]:
# Merging all three datasets on 'Country Name'
merged_df = pd.merge(on_off_track_df_cleaned, anc4_sab_pivot, on='Country Name', how='left')
merged_df = pd.merge(merged_df, population_df_selected, on='Country Name', how='left')

# Displaying the first few rows of the merged dataset
merged_df.head()

,Country Code_x,Country Name,U5MR Status,ANC4,SAB,Index,Variant,Notes,Location code,Country Code_y,...,Births (thousands),Births by women aged 15 to 19 (thousands),"Crude Birth Rate (births per 1,000 population)",Total Fertility Rate (live births per woman),Net Reproduction Rate (surviving daughters per woman),Mean Age Childbearing (years),Sex Ratio at Birth (males per 100 female births),Live Births Surviving to Age 1 (thousands),"Under-Five Deaths, under age 5 (thousands)","Under-Five Mortality (deaths under age 5 per 1,000 live births)"
0,AFG,Afghanistan,Off-Track,27.6,61.8,8142,Medium,,4,AFG,...,1 447,187.0,35.1,5,2.0,29,105.00,1397.1,76,53
1,AGO,Angola,Off-Track,NaN,NaN,3560,Medium,,24,AGO,...,1 360,254.0,38.2,5,2.3,29,103.00,1306.7,93,69
2,AIA,Anguilla,On-Track,NaN,100.0,15805,Medium,19,660,AIA,...,0,0,9.4,1,0.6,28,104.00,0.1,0,7
3,ALB,Albania,On-Track,77.8,99.8,13514,Medium,,8,ALB,...,29.0,1.0,10.1,1,0.7,29,107.00,28.6,0,9
4,AND,Andorra,On-Track,NaN,NaN,13593,Medium,,20,AND,...,1.0,0,7.2,1,0.6,32,106.00,0.6,0,6


# Extract Whole Clean and prepare data to Google Sheet

In [33]:
# Add a new sheet with a name and specify rows and columns
worksheet_title = 'Merged_Cleaned_Data'

ws = spreadsheet.worksheet(worksheet_title)
# Use the gspread_dataframe to set the DataFrame to the sheet
from gspread_dataframe import set_with_dataframe
# Set the DataFrame to the worksheet
set_with_dataframe(ws, merged_df)